In [8]:
import urllib3
from urllib.parse import urlencode
import bs4
from bs4 import BeautifulSoup
import mysql.connector

In [9]:
URL = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?serviceKey=b2AXFuhIJNJ13Rycl5fOfm3TLx3ouEbSlBwpAXEgNmZby%2FLAvMvOwdxDZCHiB2%2B34tNwOLcyMwqXA0Vs6mFvEA%3D%3D&pageNo=1&numOfRows=10000&'
DISTRICT = [11680, 11740, 11305, 11500, 11620, 11215, 11530, 11545, 11350, 11320, 11230, 11590, 11440, 11410, 11650, 11200, 11290, 11710, 11470, 11560, 11170, 11380, 11110, 11140, 11260]

def main():

    mydb = mysql.connector.connect(
      host="localhost",
      user="sungkyun",
      password="chung741",
      database="sungkyun"
    )
    mycursor = mydb.cursor()

    getData(mydb, mycursor)


In [10]:
def getData(mydb, mycursor):
    count_limit = 0
    http = urllib3.PoolManager()
    for dist in DISTRICT:
        for year in range(2021, 2022):
            print(str(dist) + ':' + str(year))
            for month in range(7, 8):
                if month < 10:
                    ymd = str(year) + '0' + str(month)
                else:
                    ymd = str(year) + str(month)
                info = URL + 'LAWD_CD=' + str(dist) + '&DEAL_YMD=' + ymd
                resp = http.request("GET", info)
                xml_obj = bs4.BeautifulSoup(resp.data,'lxml-xml')
                insert(xml_obj, mydb, mycursor)
        count_limit = count_limit + 1
        if (count_limit >= 1000):
            print("Exceeds maximum traffic: " + count_limit)
            print("current district / year / month: " + str(dist) + " / " + str(year) + " / " + str(month))
        

In [11]:
def insert(resp, mydb, mycursor):
    rows = []
    for i in resp.findAll('item'):
        try:
            price = i.거래금액.text.replace(',', '')
            built_years = i.건축년도.text
            if int(i.월.text) < 10:
                sold_date =  i.년.text+ '-0' + i.월.text
            else:
                sold_date =  i.년.text + '-' + i.월.text
            if int(i.일.text) < 10:
                sold_date = sold_date + '-0' + i.일.text
            else:
                sold_date = sold_date + '-' + i.일.text
            street_adress = i.도로명.text
            street_adress_code = i.도로명코드.text + i.도로명건물본번호코드.text
            dong = i.법정동.text
            dong_code = i.법정동시군구코드.text + i.법정동읍면동코드.text
            name = i.아파트.text
            size = i.전용면적.text
            floor = i.층.text
            rows.append((price, built_years, sold_date, street_adress, street_adress_code, dong, dong_code, name, size, floor)) 
        except:
            print("Some attribute is missing")    
    sql = 'INSERT INTO apart_price_clone (price, built_years, sold_date, street_adress, street_adress_code, dong, dong_code, name, size, floor) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
    mycursor.executemany(sql, rows)
    mydb.commit()
    print(mycursor.rowcount, "was inserted.")
    
    

In [12]:
main()

11680:2021
17 was inserted.
11740:2021
18 was inserted.
11305:2021
15 was inserted.
11500:2021
33 was inserted.
11620:2021
12 was inserted.
11215:2021
3 was inserted.
11530:2021
32 was inserted.
11545:2021
21 was inserted.
11350:2021
50 was inserted.
11320:2021
22 was inserted.
11230:2021
22 was inserted.
11590:2021
20 was inserted.
11440:2021
13 was inserted.
11410:2021
20 was inserted.
11650:2021
16 was inserted.
11200:2021
8 was inserted.
11290:2021
29 was inserted.
11710:2021
26 was inserted.
11470:2021
18 was inserted.
11560:2021
18 was inserted.
11170:2021
2 was inserted.
11380:2021
19 was inserted.
11110:2021
4 was inserted.
11140:2021
4 was inserted.
11260:2021
13 was inserted.
